In [63]:
import numpy as np
import itertools
import sympy as sp

# Define the four functions

## Bin(INT), Bin(INT) -> Bin(INT)
def identity(x):
    return x

def swap(x):
    return 1-x

def to0(_):
    return 0

def to1(_):
    return 1


basic_strategies = [identity, swap, to0, to1]
all_strategies = list(itertools.product(basic_strategies, repeat=2))

In [64]:
def generate_top(u):
    f_u, g_u = all_strategies[u]
    print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    # value 1 at point(a,b) = (f_u(x),g_u(y)), zero everywhere else
    def middle_entry(x,y):
        # value 1 at (f_u(x),g_u(y)) zero everywhere else
        middle = np.zeros((2,2), dtype=int)
        middle[f_u(x), g_u(y)] = 1
        return middle

    top_level = np.array([
                [middle_entry(0,0), middle_entry(1,0)], 
                [middle_entry(0,1), middle_entry(1,1)]
                ]) 

    return top_level


# u=5
# print("top level for strategy u =", u)
# generate_top(u)

In [65]:
# make it symbolic
tokens = [f"w{i}" for i in range(16)]
symb = sp.symbols(" ".join(tokens))

In [66]:
%%capture

# add strategy matrices
summed_strategies = sum([symb[i]*generate_top(i) for i in range(16)])

In [67]:
summed_strategies

array([[[[w0 + w10 + w2 + w8, w1 + w11 + w3 + w9],
         [w12 + w14 + w4 + w6, w13 + w15 + w5 + w7]],

        [[w10 + w4 + w6 + w8, w11 + w5 + w7 + w9],
         [w0 + w12 + w14 + w2, w1 + w13 + w15 + w3]]],


       [[[w1 + w10 + w2 + w9, w0 + w11 + w3 + w8],
         [w13 + w14 + w5 + w6, w12 + w15 + w4 + w7]],

        [[w10 + w5 + w6 + w9, w11 + w4 + w7 + w8],
         [w1 + w13 + w14 + w2, w0 + w12 + w15 + w3]]]], dtype=object)

Found summed strategies matrices for all U!

In [68]:
# declaring Gurobi variable W_i with each lb=0 and ub=1
import gurobipy as gp
from gurobipy import GRB
m = gp.Model()
W = m.addVars(16, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="W")
m.update()

In [69]:
dist_ABXY = {
    (0, 0, 0, 0): 0.0,
    (0, 0, 0, 1): 0.008333333333333333,
    (0, 0, 1, 0): 0.016666666666666666,
    (0, 0, 1, 1): 0.025,
    (0, 1, 0, 0): 0.03333333333333333,
    (0, 1, 0, 1): 0.041666666666666664,
    (0, 1, 1, 0): 0.05,
    (0, 1, 1, 1): 0.058333333333333334,
    (1, 0, 0, 0): 0.06666666666666667,
    (1, 0, 0, 1): 0.075,
    (1, 0, 1, 0): 0.08333333333333333,
    (1, 0, 1, 1): 0.09166666666666666,
    (1, 1, 0, 0): 0.1,
    (1, 1, 0, 1): 0.10833333333333334,
    (1, 1, 1, 0): 0.11666666666666667,
    (1, 1, 1, 1): 0.125}
sum(dist_ABXY.values()) == 1

# get P_AB|XY from P_ABXY
def P_AB_giv_XY(A,B, X, Y):
    P_ABXY = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B and x==X and y==Y])
    P_AB = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B])
    return P_ABXY/P_AB


# dist_AB_given_XY_2 = {
#     (0,0,0,0): 0.5,
#     (1,1,0,0): 0.5,
#     (1,0,0,0): 0,
#     (0,1,0,0): 0,
#     (0,0,0,1): 0.25,
#     (1,1,0,1): 0.25,
#     (0,1,0,1): 0.25,
#     (1,0,0,1): 0.25,
#     (0,0,1,0): 0.5,
#     (1,1,1,0): 0.5,
#     (0,1,1,0): 0,
#     (1,0,1,0): 0,
#     (0,0,1,1): 0.25,
#     (1,1,1,1): 0.25,
#     (0,1,1,1): 0,
#     (1,0,1,1): 0.25}

# # get P_AB|XY from P_ABXY
# def P_AB_giv_XY(A,B, X, Y):
#     return dist_AB_given_XY_2[A,B,X,Y]

In [70]:
np.kron(0,0)

0

In [71]:
# P(AB|XY)

# v=0 # works!
# v=1 # infeasible
# v=0.000000000001
def P_AB_giv_XY(A,B, X, Y):
    return v*(0.5*np.kron((A % 2) & B, X*Y)) + (1-v)/4



In [72]:
# summed_strategies[x,y][a,b]       # [x,y][a,b]
# P_AB_giv_XY(a,b,x,y)              # [a,b][x,y]

In [73]:
sym_to_gurobi = {symb[i]: W[i] for i in range(16)}
for x,y, a,b in itertools.product(range(2), repeat=4):
    # summed_strategies returns SymPy expression
    sympy_expr = summed_strategies[x,y][a,b]

    # Convert the SymPy expression to a Gurobi expression
    gurobi_expr = sum(sym_to_gurobi[sym] * coef for sym, coef in sympy_expr.as_coefficients_dict().items())

    # Add the constraint to the model
    m.addConstr(gurobi_expr == P_AB_giv_XY(a,b,x,y), f"P({a},{b}|{x},{y}) == {gurobi_expr} == {P_AB_giv_XY(a,b,x,y)}")


In [74]:
m.update()
constr = m.getConstrs()

## SymPy Expression

In [75]:
# summed_strategies[x,y][a,b]
# print("P(a,b|x,y)")

for x,y, a,b in itertools.product(range(2), repeat=4):
     print(f"P({a},{b}|{x},{y}) = ", summed_strategies[x,y][a,b], " = ", P_AB_giv_XY(a,b,x,y))

P(0,0|0,0) =  w0 + w10 + w2 + w8  =  0.24999999999975
P(0,1|0,0) =  w1 + w11 + w3 + w9  =  0.24999999999975
P(1,0|0,0) =  w12 + w14 + w4 + w6  =  0.24999999999975
P(1,1|0,0) =  w13 + w15 + w5 + w7  =  0.24999999999975
P(0,0|0,1) =  w10 + w4 + w6 + w8  =  0.24999999999975
P(0,1|0,1) =  w11 + w5 + w7 + w9  =  0.24999999999975
P(1,0|0,1) =  w0 + w12 + w14 + w2  =  0.24999999999975
P(1,1|0,1) =  w1 + w13 + w15 + w3  =  0.24999999999975
P(0,0|1,0) =  w1 + w10 + w2 + w9  =  0.24999999999975
P(0,1|1,0) =  w0 + w11 + w3 + w8  =  0.24999999999975
P(1,0|1,0) =  w13 + w14 + w5 + w6  =  0.24999999999975
P(1,1|1,0) =  w12 + w15 + w4 + w7  =  0.24999999999975
P(0,0|1,1) =  w10 + w5 + w6 + w9  =  0.24999999999975
P(0,1|1,1) =  w11 + w4 + w7 + w8  =  0.24999999999975
P(1,0|1,1) =  w1 + w13 + w14 + w2  =  0.24999999999975
P(1,1|1,1) =  w0 + w12 + w15 + w3  =  0.25000000000025


## Gurobi Constraints

In [76]:
for cons in constr:
    print(cons)

<gurobi.Constr P(0,0|0,0) == W[0] + W[10] + W[2] + W[8] == 0.24999999999975>
<gurobi.Constr P(0,1|0,0) == W[1] + W[11] + W[3] + W[9] == 0.24999999999975>
<gurobi.Constr P(1,0|0,0) == W[12] + W[14] + W[4] + W[6] == 0.24999999999975>
<gurobi.Constr P(1,1|0,0) == W[13] + W[15] + W[5] + W[7] == 0.24999999999975>
<gurobi.Constr P(0,0|0,1) == W[10] + W[4] + W[6] + W[8] == 0.24999999999975>
<gurobi.Constr P(0,1|0,1) == W[11] + W[5] + W[7] + W[9] == 0.24999999999975>
<gurobi.Constr P(1,0|0,1) == W[0] + W[12] + W[14] + W[2] == 0.24999999999975>
<gurobi.Constr P(1,1|0,1) == W[1] + W[13] + W[15] + W[3] == 0.24999999999975>
<gurobi.Constr P(0,0|1,0) == W[1] + W[10] + W[2] + W[9] == 0.24999999999975>
<gurobi.Constr P(0,1|1,0) == W[0] + W[11] + W[3] + W[8] == 0.24999999999975>
<gurobi.Constr P(1,0|1,0) == W[13] + W[14] + W[5] + W[6] == 0.24999999999975>
<gurobi.Constr P(1,1|1,0) == W[12] + W[15] + W[4] + W[7] == 0.24999999999975>
<gurobi.Constr P(0,0|1,1) == W[10] + W[5] + W[6] + W[9] == 0.249999999

In [77]:
# set objective, find all W_i
obj = sum(W[i] for i in range(16))
m.setObjective(obj, GRB.MAXIMIZE)
m.update()

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 16 columns and 64 nonzeros
Model fingerprint: 0x5687b77f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 3e-01]
Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Presolved: 9 rows, 16 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.015000e+00   0.000000e+00      0s
      10    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+00


In [78]:
print("***********************")
for i in range(16):
    print(f"W_{i} = {W[i].x}")

***********************
W_0 = 5.000166947155549e-13
W_1 = 0.0
W_2 = 0.0
W_3 = 0.0
W_4 = 0.0
W_5 = 0.0
W_6 = 0.0
W_7 = 0.0
W_8 = 0.24999999999975
W_9 = 0.24999999999975
W_10 = -5.000166947155549e-13
W_11 = 0.0
W_12 = 0.24999999999975
W_13 = 0.24999999999975
W_14 = 0.0
W_15 = 0.0
